In [1]:
import pandas as pd
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import re
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
!pip install eunjeon
from eunjeon import Mecab

  Created wheel for eunjeon: filename=eunjeon-0.4.0-cp38-cp38-win_amd64.whl size=35014968 sha256=678d30fa3f6b7fceedeaf27597fe02c90e3ff8411e1e0d86ab34466f0da28121
  Stored in directory: c:\users\ad\appdata\local\pip\cache\wheels\4b\1d\e9\bfa4c18245fc292aaa650fe6dd982a4feddfaab49f3b8f9f25
Successfully built eunjeon


In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x1be4aa94070>)

In [4]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data))

전체 리뷰 개수 : 200000


In [5]:
total_data['label'] = np.select([total_data.ratings > 3], [1], default=0)
total_data[:5]

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [6]:
total_data.drop_duplicates(subset=['reviews'], inplace=True)
print('총 샘플의 수 :',len(total_data))

총 샘플의 수 : 199908


In [7]:
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

훈련용 리뷰의 개수 : 149931
테스트용 리뷰의 개수 : 49977


In [8]:
# 한글과 공백을 제외하고 모두 제거
train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['reviews'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

ratings    0
reviews    0
label      0
dtype: int64


<ipython-input-8-5c91c21161d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\ad\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [9]:
test_data.drop_duplicates(subset = ['reviews'], inplace=True) # 중복 제거
test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['reviews'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 49977


<ipython-input-9-5968a3f5f2bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop_duplicates(subset = ['reviews'], inplace=True) # 중복 제거
<ipython-input-9-5968a3f5f2bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행


In [10]:
mecab = Mecab()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '것', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
train_data['tokenized'] = train_data['reviews'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_data['tokenized'] = test_data['reviews'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

<ipython-input-11-2ccd7617a3d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['tokenized'] = train_data['reviews'].apply(mecab.morphs)
<ipython-input-11-2ccd7617a3d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])


In [ ]:
negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)

In [13]:
negative_word_count = Counter(negative_words)
print(negative_word_count.most_common(20))

[('네요', 31823), ('는데', 20095), ('안', 19748), ('어요', 14869), ('있', 13200), ('너무', 13056), ('했', 11796), ('좋', 9797), ('배송', 9623), ('같', 8995), ('거', 8904), ('어', 8896), ('구매', 8884), ('없', 8695), ('아요', 8636), ('습니다', 8436), ('그냥', 8353), ('되', 8350), ('잘', 8029), ('않', 7986)]


In [14]:
positive_word_count = Counter(positive_words)
print(positive_word_count.most_common(20))

[('좋', 39365), ('아요', 21151), ('네요', 19906), ('어요', 18672), ('잘', 18612), ('구매', 16175), ('습니다', 13322), ('있', 12380), ('배송', 12263), ('는데', 11579), ('합니다', 9850), ('했', 9806), ('먹', 9453), ('재', 9252), ('너무', 8398), ('같', 7867), ('만족', 7225), ('거', 6485), ('기', 6341), ('어', 6317)]


In [15]:
X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

In [16]:
#토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [17]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
        

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 39740
등장 빈도가 1번 이하인 희귀 단어의 수: 18056
단어 집합에서 희귀 단어의 비율: 45.4353296426774
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.7909317935219462


In [18]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)
print(X_train)

단어 집합의 크기 : 21686
[list(['사이즈', '센치', '씩', '늘린', '건데', '작', '아요', '그리고', '색상', '완전', '달라요', '칙칙', '핑크', '네요', 'ㅠㅠ', '많이', '아쉽', '지만', '암막', '효과', '좋', '아요'])
 list(['ㅂ', '불', '만족', '빗이', '아픔', '멍', '피부', '빗', '질', '못해', '주', '겟', '네요'])
 list(['제품', '쓰', '삼', '일', '만', '변기', '물', '잘', '안', '내려갔', '어요', '혹시나', '해서', '다시', '빼', '보', '니', '물', '다시', '잘', '내려가', '네요', '많', '걸', '어쩌', '나요', '반품', '싶'])
 ... list(['장', '주문', '안', '됩니다', '장', '가능', '해요'])
 list(['하림', '치킨', '여기', '서', '구입', '니', '엄청', '저렴', '네요', '배송', '쾅', '꽝', '얼', '어서', '도착', '아주', '만족', '합니다', 'ㅋㅋ'])
 list(['조금', '약해', '보이', '는데', '저렴', '잘', '삿', '어요'])]


In [19]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(X_train[:3])
print(X_test[:3])

[[66, 2066, 300, 14196, 260, 72, 6, 237, 169, 134, 806, 2938, 623, 2, 75, 61, 205, 39, 1355, 154, 3, 6], [478, 404, 52, 8472, 2596, 2372, 337, 2939, 247, 2386, 38, 471, 2], [43, 24, 816, 102, 34, 2373, 159, 7, 10, 8006, 4, 1325, 29, 138, 320, 44, 58, 159, 138, 7, 1934, 2, 112, 160, 1387, 301, 119, 133]]
[[14, 710, 771, 117, 185, 248, 12], [337, 3901, 61, 3816, 1606], [11, 69, 2, 48, 164, 3, 15, 6, 512, 288, 17, 92, 109, 584, 58, 7, 2]]


In [20]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이 : 85
리뷰의 평균 길이 : 15.226184044660544


In [21]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 80
below_threshold_len(max_len, X_train)
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

전체 샘플 중 길이가 80 이하인 샘플의 비율: 99.99933302652553


In [22]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
2000/2000 [==============================] - 102s 50ms/step - loss: 0.3240 - acc: 0.8685 - val_loss: 0.2460 - val_acc: 0.9140

Epoch 00001: val_acc improved from -inf to 0.91396, saving model to best_model.h5
Epoch 2/15
2000/2000 [==============================] - 98s 49ms/step - loss: 0.2173 - acc: 0.9210 - val_loss: 0.2136 - val_acc: 0.9237

Epoch 00002: val_acc improved from 0.91396 to 0.92367, saving model to best_model.h5
Epoch 3/15
2000/2000 [==============================] - 100s 50ms/step - loss: 0.2002 - acc: 0.9268 - val_loss: 0.2035 - val_acc: 0.9261

Epoch 00003: val_acc improved from 0.92367 to 0.92610, saving model to best_model.h5
Epoch 4/15
2000/2000 [==============================] - 101s 50ms/step - loss: 0.1868 - acc: 0.9340 - val_loss: 0.2080 - val_acc: 0.9254

Epoch 00004: val_acc did not improve from 0.92610
Epoch 5/15
2000/2000 [==============================] - 103s 51ms/step - loss: 0.1782 - acc: 0.9366 - val_loss: 0.2053 - val_acc: 0.9266

Epoch 000

In [23]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1562/1562 [==============================] - 11s 7ms/step - loss: 0.2100 - acc: 0.9254

 테스트 정확도: 0.9254


In [24]:
positive_data = []
negative_data = []

def sentiment_predict(new_sentence):
  print(new_sentence)
  sentence_noun = mecab.nouns(new_sentence)
    
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    positive_data.append(sentence_noun)
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
    negative_data.append(sentence_noun)

In [25]:
#이 아래는 예시 리뷰

In [26]:
df_data = pd.read_csv('바인드 백팩 블랙_최종_.csv', index_col = 0)

In [27]:
df_data

,별점,리뷰내용
0,10,역시 마크곤잘레스..❤️ 사랑해요ㅠㅠ백팩 너무 예뻐요ㅠㅠ 엔젤키링은 조금 유치하지 ...
1,10,가방 크기도 적당하고 언제든 들고 다니기 편한것 같습니다!
2,10,수납공간도 넓고 학교책가방용으로 샀는데 디자인도 이뻐서 추천해요!!
3,10,학생가방으로 딱이에요 다른가방에 비해 긴편이긴한데 길어서 오히려 이쁜 것같다고 하네...
4,10,무난무난하게 잘메고다닐거같아요 추천합니다
...,...,...
796,10,근데 예뻐요 키가 161인데 엉덩이까지 오구 등짝을 다 덮어요 귀엽고 수납 공간 많...
797,10,진짜러 수납공간크고 예쁘고 너무 잘산거같다는생각듭ㅂ니다
798,10,좋아요\n커서 많이 들어가고 3개로 나누어져 있어서 편하고 예뻐요
799,10,다른 브랜드 가방이랑 고민하다가 할머니도 이게 더 예쁘다고 하셔서 바로 이거로 구매...


In [28]:
df_data['리뷰내용'] = df_data['리뷰내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_data['리뷰내용'].replace('', np.nan, inplace=True)
print(df_data['리뷰내용'])

0      역시 마크곤잘레스 사랑해요ㅠㅠ백팩 너무 예뻐요ㅠㅠ 엔젤키링은 조금 유치하지 않을까 ...
1                        가방 크기도 적당하고 언제든 들고 다니기 편한것 같습니다
2                    수납공간도 넓고 학교책가방용으로 샀는데 디자인도 이뻐서 추천해요
3      학생가방으로 딱이에요 다른가방에 비해 긴편이긴한데 길어서 오히려 이쁜 것같다고 하네...
4                                 무난무난하게 잘메고다닐거같아요 추천합니다
                             ...                        
796    근데 예뻐요 키가 인데 엉덩이까지 오구 등짝을 다 덮어요 귀엽고 수납 공간 많아서 ...
797                       진짜러 수납공간크고 예쁘고 너무 잘산거같다는생각듭ㅂ니다
798                    좋아요커서 많이 들어가고 개로 나누어져 있어서 편하고 예뻐요
799    다른 브랜드 가방이랑 고민하다가 할머니도 이게 더 예쁘다고 하셔서 바로 이거로 구매...
800    진짜 너무 예쁘고 무엇보다 튼튼한게 느껴지고 수납공간이 착착 있는건 아니지만 잘 나...
Name: 리뷰내용, Length: 801, dtype: object


In [29]:
lists = np.array(df_data['리뷰내용'].tolist())
print(lists[1])

가방 크기도 적당하고 언제든 들고 다니기 편한것 같습니다


In [126]:
for i in range (len(lists)):
    sentiment_predict(lists[i])
    print("")

역시 마크곤잘레스 사랑해요ㅠㅠ백팩 너무 예뻐요ㅠㅠ 엔젤키링은 조금 유치하지 않을까 라는 생각이였는데 절때 아니예요 엔젤키링이 있어야 확 사는 느낌이더라구요 앞으로 쭉 이 백팩만 매고 다닐 예정입니당 가방 고민하고 계시면 무조건 마크곤잘레스 추천드립니다
91.77% 확률로 긍정 리뷰입니다.

가방 크기도 적당하고 언제든 들고 다니기 편한것 같습니다
50.92% 확률로 부정 리뷰입니다.

수납공간도 넓고 학교책가방용으로 샀는데 디자인도 이뻐서 추천해요
68.07% 확률로 긍정 리뷰입니다.

학생가방으로 딱이에요 다른가방에 비해 긴편이긴한데 길어서 오히려 이쁜 것같다고 하네요메고 다닐때도 불편한 건 없어서 괜찮아요천사일명 바나나인형도 포인트로 생각보다 너무 튀어보이진 않고 귀여워요
90.47% 확률로 긍정 리뷰입니다.

무난무난하게 잘메고다닐거같아요 추천합니다
55.73% 확률로 부정 리뷰입니다.

예쁘고 좋아요 마감도 짱짱하고 안에도 넉넉해요
69.57% 확률로 부정 리뷰입니다.

진짜 가성비 너무 좋고 디자인구성가격 전부다 너무 좋아요 실물이 특히 너무 좋습니다
93.40% 확률로 긍정 리뷰입니다.

실제로 받으니 더 만족입니다 수납 공간도 넉넉하고 다양한 스타일에 활용할 수 있어서 더 좋아요
59.76% 확률로 긍정 리뷰입니다.

수납공간도 넓고 예쁘구 친구들도 다 가방 예뻐서 좋겟다 라구 해요
69.25% 확률로 긍정 리뷰입니다.

생각보다 만족스러운 제품 적당한 수납공간과 가성비 어떻게 입든 다 잘어울리는 가방 
89.39% 확률로 긍정 리뷰입니다.

디자인이 이쁘고 포인트도 있어서 좋아요 바나나는 너무 귀여워서 누가 가져갈까봐 가방에 넣고 다님니다
70.16% 확률로 긍정 리뷰입니다.

등이 딴딴하고 크기도 너무 크지않아서 좋은 것 같아요
76.95% 확률로 부정 리뷰입니다.

노트북이 인치라 들어갈까 걱정했는데 넉넉하게 들어가요
82.22% 확률로 부정 리뷰입니다.

그냥 캐쥬얼에 딱 매고 다니기 좋고 넣고싶은거 다 넣어도 또 들어가는 마법 
98.79

In [176]:
# positive_data 를 한 배열에 모두 넣고 명사로 분리
text3
text4 = []
text5 = []
bl=[]
for i in range (len(positive_data)):
    text3 = ' '.join(positive_data[i])
    text4.append(text3)
text5 = ' '.join(text4)

print(text5)


마크곤잘레스 사랑 엔젤 키 링 생각 절 때 엔젤 키 링 느낌 앞 이 백 팩 예정 방 고민 마크곤잘레스 추천 가방 크기 언제 것 수납 공간 학교 책가방 디자인 추천 학생 가방 가방 편 것 때 불편 건 천 사 일 명 바나나 인형 포인트 생각 거 추천 마감 안 진짜 성비 디자인 구성 가격 실물 만족 수납 공간 스타일 활용 수 수납 공간 친구 가방 생각 만족 제품 수납 공간 과 성비 가방 디자인 포인트 바나나 누가 가방 등 것 노트북 인치 걱정 쥬얼 거 마법 유쾌 쾌 중 남아 요요 고요 학년 때 가방 달 거북 등딱지 느낌 가방 수납 공간 달기 짜증 맘 울딸 완전 마음 등산 가방 스타일 처음 앞 주머니 수납 공간 게 단점 것 안정 완전 만족 노트북 인치 가방 이상 가방 등 부분 판 위 쿠션 등 가방 가방 수납 공간 디자인 교복 매도 대 거 인형 키 걱정 크기 만족 여성 거 디자인 걱정 가방 데 만큼 인형 최고 포인트 책 일다 수납 공간 편리 디자인 옷 때 구매 만족 수납 공간 인형 포인트 여러분 마크 곤잘레스 수납 공간 여 인형 가방 디자인 인형 가방 최고 패션 아이템 아기 자기 가방 수 여 학생 여행 레저 적극 추천 남자 생각 생각 용 구매 하세오 완존 귀여 웅 가방 수납 공간 인형 맘 옷 수납 공간 편 무엇 엔젤 마음 키 센티 지퍼 스트링 지퍼 엔젤 키 링 가방 수납 공간 최고 사이즈 수납 공간 마크곤잘레스 인형 티머니 카드 디자인 수납 공간 칸 바나나 줄 천사 인형 거 노랑 색 인트 거 글씨체 바나나 포인트 바나나 인형 앞 배송 것 갗 수납 공간 낭 낭 인형 카드 생각 것 거 강추 바나나 맘 거 불편 가방 수납 공간 여 디자인 여 실용 송 가방 가방 밑부분 받침 데 쿠션 처리 것 수납 칸 칸 쉬망 오거 나이저 중간 칸 거 개 수납 전체 만족 걸 원 생각 게 여행 것 키 링 마음 사은품 마음 전체 조화 유행 느낌 내구 앞 기대 가방 용 가방 공간 옷 보따리장수 왕 가방 로고 생각 수납 공간 스티커 노트북 로고 빛 반사 것 생각 용량 가방 용량 마음 디자인 디자인 실

In [182]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    mecab = Mecab()  # 형태소 추출
    nouns = mecab.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns


vect = CountVectorizer(tokenizer = lambda x: text_cleaning(text5))
print(vect)
#word_list = vect.get_feature_names()
#count_list = bow_vect.toarray().sum(axis=0)


ValueError: Iterable over raw text documents expected, string object received.

In [174]:
# negative_data 를 한 배열에 모두 넣고 명사로 분리
text7 = []
text8 = []
bl=[]
for i in range (len(negative_data)):
    text6 = ' '.join(negative_data[i])
    text7.append(text6)
text8 = ' '.join(text7)
print("")
nl = mecab.nouns(text8)

counter1 = Counter(nl)

# 1글자 제거
available_counter1 = Counter({x: counter1[x] for x in counter1 if len(x) > 1})
available_counter1.most_common(15)


[('가방', 159),
 ('수납', 87),
 ('공간', 77),
 ('디자인', 54),
 ('만족', 45),
 ('사이즈', 28),
 ('생각', 27),
 ('인형', 27),
 ('선물', 25),
 ('마음', 23),
 ('바나나', 22),
 ('크기', 20),
 ('구매', 19),
 ('고민', 17),
 ('노트북', 17)]

In [149]:
print(positive_data)

1


In [32]:
print(negative_data)

[['디자인', '자체', '가방', '자체', '크기'], ['디자인', '수납', '공간', '데', '물건', '주머니', '하나', '후기', '가방끈', '말'], ['산지', '얼마', '가방끈', '그게', '건'], ['달기', '환불'], ['수납', '공간', '낭', '낭', '모양'], ['생각', '가법', '막'], ['상품', '때', '상품', '냄새', '나', '머리', '수납', '공간', '생각', '디자인'], ['건', '내구', '책', '데', '이틀', '만', '가방끈', '가방', '부분', '일부', '가방', '사이즈', '가방', '물건', '것', '감안'], ['등', '감출', '디자인', '지퍼', '안쪽', '부직포', '마감', '처리', '별'], ['텍', '정도'], ['수납', '공간', '부족', '후기', '저', '부족', '느'], ['후기', '엉덩이', '글', '엉덩이', '불편', '것', '고민', '저', '제', '원', '크기', '엉덩이', '치지', '일반', '백', '팩', '크기', '것', '키', '엉덩이', '위', '가방', '개', '아무', '거'], ['내구', '것', '로고'], ['디자인', '수납', '공간', '데', '물건', '주머니', '하나', '후기', '가방끈', '말'], ['엔젤', '워', '정신'], ['생각', '넓이', '높이', '단', '단', '듯', '법', '과목', '권', '높이', '책', '곳', '두께', '듯', '가방', '사람', '수납', '공간', '책', '곳', '가방', '목적', '본질', '것', '번', '매쉬', '칸', '번', '칸', '수납', '번', '노트북', '수'], ['인치', '노트북', '수납', '가능', '인치', '무리', '것'], ['상품', '때', '상품', '냄새', '나', '머리', '수납', '공간', '생각', '디자인'], ['생각', 

In [113]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(positive_data)

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합(vocabulary)의 크기 : 721
등장 빈도가 1번 이하인 희귀 단어의 수: 353
단어 집합에서 희귀 단어의 비율: 48.95977808599168
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 7.367981632227092
단어 집합의 크기 : 370


In [115]:
sentiment_predict('이 상품 진짜 좋아요... 저는 강추합니다. 대박')

이 상품 진짜 좋아요... 저는 강추합니다. 대박
83.90% 확률로 긍정 리뷰입니다.


In [ ]:
sentiment_predict('학생가방으로 딱이에요 다른가방에 비해 긴편이긴한데 길어서 오히려 이쁜 것같다고 하네요메고 다닐때도 불편한 건 없어서 괜찮아요천사(일명 바나나)인형도 포인트로 생각보다 너무 튀어보이진 않고 귀여워요')

In [69]:
sentiment_predict('디자인은 마음에 들어요. 지퍼는 조금 불편할것 같아요!')

92.48% 확률로 긍정 리뷰입니다.


In [73]:
sentiment_predict('오버핏이라고 생각했지만 생각한것보다 더  크네요!')

55.55% 확률로 부정 리뷰입니다.


In [74]:
sentiment_predict('발볼이 너무 조여서 조금 불편하지만 예뻐서 만족합니다 ㅎㅎ')

94.46% 확률로 긍정 리뷰입니다.


In [75]:
sentiment_predict('괜찮아요 근데 발볼이 좀 좁고 길이는̆̈ 좀 커요')

81.09% 확률로 긍정 리뷰입니다.


In [76]:
sentiment_predict('색깔이 너무 밝고 사진이랑 너무 달라요. 환불하고싶은데 택을 떼버려서 못하는겁니다 다른색깔사세요')

99.90% 확률로 부정 리뷰입니다.


In [77]:
sentiment_predict('이쁘긴 한데 얇고 밑에가 늘어나서 핏이 안 이뻐져요 원단도 가격에 비해 좋지 않아서 아쉽네요')

96.10% 확률로 부정 리뷰입니다.


In [79]:
sentiment_predict('사이즈 xs s 고민하다가 s 샀는데 그렇게 크다고는 못느끼겠어요 저보다 크신분들이 저랑 같은 사이즈 사시는거 보고 조금 걱정했는데 너무 크지도 않고 그냥 적당히 낙낙해서 입기 좋아요 제 기준 엉덩이 다 덮어요 그리고 뻣뻣한 재질이라 좋았어요 근데 이 가격치곤...실밥도 많이 나오고..')

95.70% 확률로 긍정 리뷰입니다.


In [81]:
sentiment_predict('디네뎃답지않게거의정사이즈에가깝네요')

61.83% 확률로 부정 리뷰입니다.


In [83]:
sentiment_predict('기본티임 특별할것도 없지만 그렇다고 안좋지도 않음')

91.95% 확률로 부정 리뷰입니다.
